In [1]:
import findspark
findspark.init("/usr/local/Cellar/apache-spark/3.1.2/libexec")
import pyspark
from pyspark.sql import SparkSession

from pyspark.sql.functions import*
from pyspark.sql.types import*
import time

import os

import numpy as np
import re
import pandas as pd 
import traceback
import sys
from time import sleep
import traceback
import smtplib


In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
except:
    print('Something went wrong...')

In [4]:
########## Function to Send Email #########

def sendmail(sub, body):
    try:
       SERVER = "us.relay.ibm.com"
       #SERVER = "smtp.gmail.com"
       #SERVER = smtplib.SMTP('smtp.gmail.com', 587)
       #SERVER.ehlo()
       FROM = "romdbload@radial.com"
       TO = ["mlakshm@us.ibm.com"] # must be a list
       SUBJECT = sub
       TEXT = body
       message = 'From:'+FROM+\
       '\nTo: '+", ".join(TO)+\
       '\nSubject: '+SUBJECT+'\n'+TEXT
       print(message)
       server = smtplib.SMTP(SERVER)
       server.sendmail(FROM, TO, message)
       server.quit()
       return 0
    except Exception as e:
       exp_tb=traceback.format_exc()
       print(exp_tb)


In [5]:
sendmail("Test","Test")

From:romdbload@radial.com
To: mlakshm@us.ibm.com
Subject: Test
Test
Traceback (most recent call last):
  File "<ipython-input-4-b98671e32813>", line 17, in sendmail
    server = smtplib.SMTP(SERVER)
  File "/Users/mlakshm/opt/anaconda3/lib/python3.8/smtplib.py", line 253, in __init__
    (code, msg) = self.connect(host, port)
  File "/Users/mlakshm/opt/anaconda3/lib/python3.8/smtplib.py", line 339, in connect
    self.sock = self._get_socket(host, port, self.timeout)
  File "/Users/mlakshm/opt/anaconda3/lib/python3.8/smtplib.py", line 308, in _get_socket
    return socket.create_connection((host, port), timeout,
  File "/Users/mlakshm/opt/anaconda3/lib/python3.8/socket.py", line 787, in create_connection
    for res in getaddrinfo(host, port, 0, SOCK_STREAM):
  File "/Users/mlakshm/opt/anaconda3/lib/python3.8/socket.py", line 918, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 8] nodename nor servname provided, or not 

In [6]:
tbllist=[]
jsonpathlists=[]
colslist=[]

with open('json_column_map.txt') as f:
    contents = f.read()
    contents=contents.strip()
    maplist=contents.split("TABLE_NAME:")
    #print(maplist)
    #maplist = list(filter([], maplist))
    for lmap in maplist:
        lmaplist=lmap.split("\n")
        #lmaplist = list(filter(None, lmaplist))
        tbl=lmaplist.pop(0)
        tbllist.append(tbl)
        collist=[]
        jsonpathlist=[]
        for col_json in lmaplist:
            coljsonarr=col_json.split(",")
            if (len(coljsonarr)==2):
               collist.append(coljsonarr[0])
               jsonpathlist.append(coljsonarr[1])
        colslist.append(collist)
        jsonpathlists.append(jsonpathlist)
            
            
f.close()


tbllist = [x for x in tbllist if x != 'KAFKARADIAL.']
tbllist = [x for x in tbllist if x != '']
colslist = [x for x in colslist if x != []]
jsonpathlists = [x for x in jsonpathlists if x != []]


print(tbllist[6])
print(jsonpathlists[6])
print(colslist[6])

KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0
['order.lineItems.0.charges.0.taxes.0.jurisdictionLevel', 'order.id', 'order.lineItems.0.charges.0.taxes.0.situs', 'order.lineItems.0.charges.0.taxes.0.impositionValue', 'order.sellerId', 'order.lineItems.0.charges.0.taxes.0.impositionType', 'order.lineItems.0.charges.0.taxes.0.jurisdictionId']
['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']


In [7]:

##### Input Tables ######

#tbllist=['KAFKARADIAL.ORDER_HEADER_STG', 'KAFKARADIAL.ORDER_LINE_STG', 'KAFKARADIAL.ORDER_CHARGES_STG']

In [8]:

##### Column Mapping ######

#jsonpathlists=[['msg.order.orderTotal.promotionCharge', 'msg.order.id', 'msg.order.sellerId', 'msg.order.orderTotal.tax', 'msg.order.orderTotal.total', 'msg.order.orderTotal.adjustmentCharge', 'msg.order.type'], ['msg.order.id', 'msg.order.lineItems.0.quantity', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.color', 'msg.order.lineItems.0.lineTotal.totalTaxAmount', 'msg.order.lineItems.0.unitPrice', 'msg.order.lineItems.0.lineTotal.totalPromotionChargeAmount', 'msg.order.lineItems.0.lineTotal.netChargeAmount', 'msg.order.lineItems.0.itemDetail.itemId', 'msg.order.lineItems.0.lineTotal.totalAdjustmentChargeAmount', 'msg.order.lineItems.0.itemDetail.itemDesc', 'lookup'], ['msg.order.id', 'msg.order.lineItems.0.charges.0.id', 'msg.order.lineItems.0.lineNo', 'msg.order.lineItems.0.charges.0.detail.amount', 'msg.order.lineItems.0.charges.0.name']]

In [9]:

###### Columns of the Tables #####

#colslist=[['ORDER_PROMO_AMT', 'ORDER_ID', 'SELLER_ID', 'ORDER_TAX_AMT', 'ORDER_TOTAL_AMT', 'ORDER_ADJ_AMT', 'ORDER_TYPE'], ['ORDER_ID', 'ITEM_QTY', 'LINE_NUM', 'COLOR', 'LINE_TAX_AMT', 'ITEM_AMT', 'LINE_PROMO_AMT', 'LINE_TOTAL_AMT', 'ITEM_ID', 'LINE_ADJ_AMT', 'ITEM_DESC', 'COUNTRY_OF_ORIGIN'], ['ORDER_ID', 'CHARGE_ID', 'LINE_NUM', 'CHARGE_AMOUNT', 'CHARGE_NAME']]

In [10]:

########## Reading Input Stream in JSON Format from Kafka ###########

try:

 df = spark \
  .readStream \
  .option("multiLine", True).option("mode", "PERMISSIVE") \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "test3") \
  .load() \
  .selectExpr("CAST(value AS STRING)")

except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)
    

In [11]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
###WASHINGTON

jdbcUrl="jdbc:db2://67beb010-4100-48d1-a281-7b5af97791aa.bs2ipa7w0uv9tsomi9ig.databases.appdomain.cloud:31815/bludb:sslConnection=true;"
user="a5406f1c"
password="eK1EhADAU7y3W61T"

In [13]:
###### Lookup Country of Origin #######

key ="'100'"
keyval="SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID="+key

print(keyval)
dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+keyval+') as t').option('user', user).option('password', password).load()
dbl.show()


SELECT COUNTRY_OF_ORIGIN FROM KAFKARADIAL.PRODUCT_MASTER where ITEM_ID='100'
+-----------------+
|COUNTRY_OF_ORIGIN|
+-----------------+
+-----------------+



In [14]:
def writeToSQLWarehouse(df,epochId,tbl):
  print(tbl)
  print(df)
  df.write.format("jdbc") \
  .mode("append") \
  .option("url", jdbcUrl) \
  .option("dbtable", tbl) \
  .option("user", user) \
  .option("password", password) \
  .save()

In [15]:
def prepMultiRows(df,epochID):
    df.show()
    l = []
    for row in df.rdd.collect():
       x=row[0]
       dfr = spark.read.option("multiline", "true") \
       .json(spark.sparkContext.parallelize([x]))
       dfr.show()
       l.append(dfr)
    print("***")

    return l

In [16]:
def popTables(df, epochId):
    
 try:  
    
   if df.rdd.isEmpty():
     print(df.rdd.isEmpty)
   else:
    print("before")
    #df=df.withColumnRenamed("from_json(CAST(value AS STRING))", "msg")
    dflist=prepMultiRows(df,epochId)
    print("after")
    
    # Create an empty RDD
    emp_RDD = spark.sparkContext.emptyRDD()
 
    # Create empty schema
    columns = StructType([])
 
    # Create an empty RDD with empty schema
    dfblk = spark.createDataFrame(data = emp_RDD, schema = columns)
    
    i=0
    for jmsg in dflist:
        jmsg.show()
        #print(jsonpathlist)
        print("****")
        if ( jmsg.columns[0] != '_corrupt_record' ):
            if i==0:
                dfblk=jmsg
                i=i+1
            else:
                dfblk.union(jmsg)  
            
        else:
            print("Corrupt Record:")
            jmsg.show()
            
    prepareExpDF(dfblk,epochId)
    
 except Exception as e:
    
    exp_tb=traceback.format_exc()
    sendmail("ERROR",exp_tb)
    print(exp_tb)


In [17]:
###### Data Formatting for JDBC Load #######

def formatVal(val,jsonpathval):

    if val== False or val =='false':
       jsonpathval.append("0")
    elif val is None:
       jsonpathval.append("None")
    elif isinstance(val, (int,np.integer)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    elif isinstance(val, (float,np.float64)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    else:
       jsonpathval.append(val) 


In [18]:
def formatValElm(elm,val,jsonpathval,orderid):
    

    if elm== "orderid" and orderid=="1":
       val = val.replace("-","")
    elif val is None:
       jsonpathval.append("None")
    elif isinstance(val, (int,np.integer)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    elif isinstance(val, (float,np.float64)):
       #val = val.item()
       jsonpathval.append(str(val).strip())
       #jsonpathval.append(val)
    else:
       jsonpathval.append(val) 



In [19]:
lookuptablelist=[["KAFKARADIAL.PRODUCT_MASTER","COUNTRY_OF_ORIGIN"]]

In [20]:
###### Prepare Dataframe and Populate the DB #######

## [[PRODUCT_MASTER,COUNTRY_OF_ORIGIN],[<table2>,<field2>]]

def lookupItemID(lookuptablelist, item_id):
    
    try:
       
     for lookuptable in lookuptablelist:
            
            lookuptblname=lookuptable[0]
            lookuptblfield=lookuptable[1]
            item_id="'"+item_id+"'"
            print(item_id)
            qry='SELECT '+lookuptblfield+' FROM '+lookuptblname+' WHERE ITEM_ID = '+item_id
            print(qry)
            dbl = spark.read.format('jdbc').option('url', jdbcUrl).option('dbtable', '('+qry+') as t').option('user', user).option('password', password).load()
            dbl.show()
            ctry=dbl.collect()[0][0]
            return ctry

       #### Create Dataframe with JSON value list parsed as the data and the Table Column Names as the Schema ####
       #sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    
    
    except Exception as e:
        print("Exception:")
        print(e)
        exp_tb=traceback.format_exc()
        sendmail("ERROR",exp_tb)
        #print(exp_tb)

In [21]:
#ctry=lookupItemID(lookuptablelist,'100')

In [22]:
import builtins
round = getattr(builtins, "round")

def updateData(jsonvallist,romlistfinal1,tbl):
    
    dict_from_list = dict(zip(romlistfinal1, jsonvallist))
    #print(dict_from_list["ORDER_ID"])
    
    if dict_from_list["ORDER_ID"]=="30002217661":
       amt=round(float(dict_from_list["CHARGE_AMOUNT"]),1)
       dict_from_list["CHARGE_AMOUNT"]=amt
        
    return list(dict_from_list.values())
    #return sdf

In [23]:
jsonvallist=["30002217661","5.017"]
romlistfinal1=["ORDER_ID","CHARGE_AMOUNT"]
jsonvallist=updateData(jsonvallist,romlistfinal1,"TEST")
print(jsonvallist)

['30002217661', 5.0]


In [24]:
import builtins
round = getattr(builtins, "round")
from pyspark.sql.functions import lit

def updateDF(sdf,tbl):
    
    sdf.printSchema()
    if tbl=="KAFKARADIAL.ORDER_CHARGES_STG":
       ordid=sdf.select("ORDER_ID").collect()[0][0]
     
       if ordid=="30002217661":
           chargeamt=sdf.select("CHARGE_AMOUNT").collect()[0][0]
           charge=float(chargeamt)
           charge=round(charge, 1)
           chargeamt1=str(charge)
           sdf=sdf.withColumn("CHARGE_AMOUNT",lit(chargeamt1) )

    
    return sdf
    

In [27]:
def createDF(jsonvallist,romlistfinal1):
    schema = StructType([])
    sdf = spark.createDataFrame([()], schema)
    try:
           sdf=spark.createDataFrame(data = jsonvallist, schema = romlistfinal1)
    except Exception as e:
           s = "Error {0}".format(str(e))
           if s.find("Unable to infer the type of the field") >-1:
              sarr=s.split(" ")
             
              fieldnm=sarr[-1]
              fieldnmarr=fieldnm.split(".")
              fieldclean=fieldnmarr[0]
              idxrom=romlistfinal1.index(fieldclean)
              print(idxrom)
              print(jsonvallist)
              idxjsonval=jsonvallist[0][idxrom]
              print(idxjsonval)
              tupleX=jsonvallist[0]
              tupleX = [x for x in tupleX if x!=idxjsonval]
              jsonvallist=[tupleX]
              romlistfinal1.remove(fieldclean)
              print(tupleX)
              print(romlistfinal1)
              sdf=createDF(jsonvallist,romlistfinal1)
              createDF(jsonvallist,romlistfinal1)
           else:
            print(e)
            print(jsonvallist)
            print(romlistfinal1)
    sdf.show()
    return sdf

    

In [29]:
def explodeArrays(elm,df, elmstr, elmsplitstr):
            
            elmsplitar=elmsplitstr.split(".0.")
            elmsplitarr=elmsplitar[:-1]
            elmstrprep=""
            ctr=0
            elmarrstr=""
            # Example elm: msg.order.lineItems.0.charges.0.id


            for e in elmsplitarr:
                ### examples e: msg.order.lineItems, charges
                esparr=e.split()
                e=esparr[0]
                e=e.replace('"', '')
                
                if ctr==0:
                    elmstrprep=e
                    elmarrstr=elmstrprep
                    
                    elmarrstr=re.sub('[.]','_',elmarrstr)
                    print(e,"**",elmarrstr)
                    ctr=ctr+1
                else:
                   
                   elmarrstr=elmstrprep+"_"+e
                   elmstrprep=elmstrprep+"."+e

                if (elmarrstr not in df.columns):
                   arrchk=df.select(elmstrprep).collect()[0][0]
                   if arrchk:
                      df=df.withColumn(elmarrstr,explode(elmstrprep))
                        
                elmstrprep=elmarrstr
                df.show()
            return df,elmarrstr,elmsplitar


In [30]:
######### Obtaining values from the incoming JSON Msg ###########

def prepareExpDF(df,epochId):

#### Iterates for every table that we will be populating ####


 for jsonpathlist,romlistfinal1,tbl in zip(jsonpathlists,colslist,tbllist):
    jsonpathwithval=[]
    jsonpathwithnoval=[]
    romlistwithval=[]
    romlistwithnoval=[]
    jsonpathval=[]
    lookuptablelist=[]
  

    ####### Find values for every column of a table #####
    
    elm_up_arr=[]

    for elm,colval in zip(jsonpathlist,romlistfinal1):
      elmstr=elm
      elmsplitstr=elm
      print(elm)
      try: 
        
        ########## If the JSON Path contains arrays, explode every array and obtain the values for the JSON Path 
                 
        if (elm.find(".0.")>-1):
            
            df,elmarrstr,elmsplitar=explodeArrays(elm,df, elmstr, elmsplitstr)
            elm_up=elmarrstr+"."+elmsplitar[-1]
            elm_up_arr.append(elm_up)
            romlistwithval.append(colval)
            print("********")
            df.show()
            
        else:
            
            elm_up_arr.append(elm)
            romlistwithval.append(colval)
     
      

      except Exception as e:
        print(e)
     
    prepareDF1(elm_up_arr,romlistwithval,epochId,tbl)
    

In [31]:
###### Prepare Dataframe and Populate the DB #######

def prepareDF1(df,elm_up_arr,romlistfinal1,epochId,tbl):
    
    try:
         
       print("&&&&&&&&&&")
    
       cols_str=','.join(elm_up_arr)
       cdf=df.select(cols_str)
       cdf.show()
        
        
       col_names_str = ','.join(romlistfinal1)
       sdf=cdf.toDF("col_names_str")
    
       print(jsonpathval)
    
       ######## Update Dataframe ########
        
       #sdf = updateDF(sdf,tbl)

       
       writeToSQLWarehouse(sdf, epochId,tbl)
    
    except Exception as e:
        print("hererrerre")
        print(e)

In [33]:
########### Loading Schema File ##############

jsondf = spark.read.json("Downloads/to_garage_20210922/order_created_test2.json", multiLine=True)
order_created_schema=jsondf.schema

In [34]:
jsondf.printSchema()

root
 |-- customAttributes: struct (nullable = true)
 |    |-- ORDER_REQUEST_ORIGIN: string (nullable = true)
 |-- order: struct (nullable = true)
 |    |-- addresses: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- addressId: string (nullable = true)
 |    |    |    |-- addressLine1: string (nullable = true)
 |    |    |    |-- addressType: string (nullable = true)
 |    |    |    |-- city: string (nullable = true)
 |    |    |    |-- countryCode: string (nullable = true)
 |    |    |    |-- email: string (nullable = true)
 |    |    |    |-- firstName: string (nullable = true)
 |    |    |    |-- lastName: string (nullable = true)
 |    |    |    |-- phone: string (nullable = true)
 |    |    |    |-- pobox: boolean (nullable = true)
 |    |    |    |-- stateOrProvinceCode: string (nullable = true)
 |    |    |    |-- zipCode: string (nullable = true)
 |    |-- archivedOrder: boolean (nullable = true)
 |    |-- billingAddressRefId: str

In [35]:
jsondf.show()

+----------------+--------------------+--------------------+
|customAttributes|               order|           topicName|
+----------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|
+----------------+--------------------+--------------------+



In [36]:
jsondf.select("order.lineItems.lineNo").show()

+------+
|lineNo|
+------+
|   [1]|
+------+



In [37]:
jsondf.select("order.lineItems.itemDetail.itemId").show()

+---------------+
|         itemId|
+---------------+
|[75-MH06-MD-BK]|
+---------------+



In [38]:
#jsondf.select("order.lineItems.charges.id").show()

In [39]:
table1='KAFKARADIAL.ROM_ORDER_HEADER_STG0'

In [40]:
#['order.id', 'order.sellerId', 'order.customerInfo.phone', 'order.customerInfo.customerId', 'order.customerOrderId', 'order.type', 'order.orderClosed', 'order.orderSourceCode']

In [41]:
collist1=['ORDER_ID', 'SELLER_ID', 'CUSTOMER_PHONE', 'CUSTOMER_ID', 'CUSTOMER_ORDER_ID', 'ORDER_TYPE', 'ORDER_CLOSED_FLAG', 'ORSO_CODE']

In [42]:
#### Obtaining Fields for Table1

def getOrderID(df):
    order_id = df.select("order.id").toPandas().values[0][0]
    return order_id

def getSellerID(df):
    seller_id = df.select("order.sellerId").toPandas().values[0][0]
    return seller_id

def getCustPhoneNum(df):
    cust_phone_num = df.select("order.customerInfo.phone").toPandas().values[0][0]
    return cust_phone_num

def getCustID(df):
    cust_id = df.select("order.customerInfo.customerId").toPandas().values[0][0]
    return cust_id

def getCustOrderID(df):
    cust_order_id = df.select("order.customerOrderId").toPandas().values[0][0]
    return cust_order_id

def getOrderType(df):
    order_type = df.select("order.type").toPandas().values[0][0]
    return order_type

def getOrderClosed(df):
    order_closed = df.select("order.orderClosed").toPandas().values[0][0]
    return order_closed

def getOrderSrcCode(df):
    order_src_code = df.select("order.orderSourceCode").toPandas().values[0][0]
    return order_src_code

In [43]:
#### Populating Table1:

def prepareExpDFRSimpl(df,epochId,table1):
    
    jsonpathval=[]
    romlistwithval=[]
    
    try:
        order_id = getOrderID(df)
        jsonpathval.append(order_id)
        romlistwithval.append("ORDER_ID")
    except Exception as e:
        print(e)
        
    try:
        seller_id = getSellerID(df)
        jsonpathval.append(seller_id)
        romlistwithval.append("SELLER_ID")
    except Exception as e:
        print(e)
        
    try:
        cust_phone_num = getCustPhoneNum(df)
        jsonpathval.append(cust_phone_num)
        romlistwithval.append("CUSTOMER_PHONE")
    except Exception as e:
        print(e)
    
    try:
        cust_id = getCustID(df)
        jsonpathval.append(cust_id)
        romlistwithval.append("CUSTOMER_ID")
    except Exception as e:
        print(e)
        
    try:
        cust_order_id = getCustOrderID(df)
        jsonpathval.append(cust_order_id)
        romlistwithval.append("CUSTOMER_ORDER_ID")
    except Exception as e:
        print(e)
        
    try:
        order_type = getOrderType(df)
        jsonpathval.append(order_type)
        romlistwithval.append("ORDER_TYPE")
    except Exception as e:
        print(e)
        
    try:
        order_closed = getOrderClosed(df)
        jsonpathval.append(order_closed)
        romlistwithval.append("ORDER_CLOSED_FLAG")
    except Exception as e:
        print(e)
        
    try:
        order_src_code = getOrderSrcCode(df)
        jsonpathval.append(order_src_code)
        romlistwithval.append("ORSO_CODE")
    except Exception as e:
        print(e)
        
    print(jsonpathval, romlistwithval)
    
    #prepareDf (jsonpathval, romlistwithval, epochID, table1)

In [44]:
 prepareExpDFRSimpl(jsondf,"111",table1)

['30002217646', 'PROUS', '1234567890', '001231057695', '001691000645870', 'SALES', False, 'EB2C'] ['ORDER_ID', 'SELLER_ID', 'CUSTOMER_PHONE', 'CUSTOMER_ID', 'CUSTOMER_ORDER_ID', 'ORDER_TYPE', 'ORDER_CLOSED_FLAG', 'ORSO_CODE']


In [45]:
table2='KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0'
###['order.lineItems.0.charges.0.taxes.0.jurisdictionLevel', 'order.id', 'order.lineItems.0.charges.0.taxes.0.situs', 'order.lineItems.0.charges.0.taxes.0.impositionValue', 'order.sellerId', 'order.lineItems.0.charges.0.taxes.0.impositionType', 'order.lineItems.0.charges.0.taxes.0.jurisdictionId']
###['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']

In [46]:

collist2=['JURISDICTION_LEVEL', 'ORDER_ID', 'TAX_SITUS', 'IMPOSITION_VALUE', 'SELLER_ID', 'IMPOSITION_TYPE', 'JURISDICTION_ID']


In [61]:
#### Obtaining Fields for Table2 

def explodeDFT2(df):
    
    elm = "order.lineItems.0.charges.0.taxes.0.jurisdictionLevel"
    df=df.withColumn("order_lineItems",explode("order.lineItems"))
    df.show()
    
    df=df.withColumn("order_lineItems_charges",explode("order_lineItems.charges"))
    df.show()
    
    df=df.withColumn("order_lineItems_charges_taxes",explode("order_lineItems_charges.taxes"))
    df.show()
    
    return df
    

def getJurisdictionLevelT2(df):
    
    elm = "order.lineItems.0.charges.0.taxes.0.jurisdictionLevel"
    
    print("JURIS LEVEL")
    juris_lvl_t2= df.select("order_lineItems_charges_taxes.jurisdictionLevel")
    juris_lvl_t2.show()
    
    return juris_lvl_t2

def getOrderIDT2(df):

    order_id_t2 = df.select("order.id").collect()
    print("ORDER_ID")
    order_id_t2 = df.select("order.id")
    order_id_t2.show()
    
    return order_id_t2

def getTaxSitusT2(df):
    elm = "order.lineItems.0.charges.0.taxes.0.situs"
    print("Tax Situs: ")
    tax_situs_t2 = df.select("order_lineItems_charges_taxes.situs")
    tax_situs_t2.show()
    
    return tax_situs_t2

def getImpositionValueT2(df):
    elm="order.lineItems.0.charges.0.taxes.0.impositionValue"
    print("Impos Val: ")
    impos_val=df.select("order_lineItems_charges_taxes.impositionValue")
    impos_val.show()
    return impos_val

def getSellerIDT2(df):
    print("SELLER_ID: ")
    seller_id=df.select("order.sellerId")
    seller_id.show()
    return seller_id

def getImpositionTypeT2(df):
    elm="order.lineItems.0.charges.0.taxes.0.impositionType"
    print("Impos Type: ")
    impos_type=df.select("order_lineItems_charges_taxes.impositionType")
    impos_type.show()
    return impos_type

def getJurisdictionID(df):
    elm="order.lineItems.0.charges.0.taxes.0.jurisdictionId"
    print("Juris ID: ")
    juris_id=df.select("order_lineItems_charges_taxes.jurisdictionId")
    juris_id.show()
    return juris_id


def getAllT2(df):
    
    cdf = df.select("order_lineItems_charges_taxes.jurisdictionLevel", "order.id", "order_lineItems_charges_taxes.situs", "order_lineItems_charges_taxes.impositionValue", "order.sellerId", "order_lineItems_charges_taxes.impositionType", "order_lineItems_charges_taxes.jurisdictionId" )
     
    return cdf


In [62]:
#### Populating Table2:

def prepareExpDFRSimplT2(df,epochId,table):
    
    df=explodeDFT2(df)

    try:
        juris_lvl_t2 = getJurisdictionLevelT2(df)
    except Exception as e:
        print(e)
        
    try:
        order_id = getOrderIDT2(df)
    except Exception as e:
        print(e)
        
    try:
        tax_situs_t2 = getTaxSitusT2(df)
        
    except Exception as e:
        print(e)
        
    try:
        impos_val = getImpositionValueT2(df)
    except Exception as e:
        print(e)
        
    try:
        seller_id = getSellerIDT2(df)
    except Exception as e:
        print(e)
        
    try:
        impos_type = getImpositionTypeT2(df)
    except Exception as e:
        print(e)
        
    try:
        juris_id = getJurisdictionID(df)
    except Exception as e:
        print(e)
    
    cdf=getAllT2(df)
    cdf.show()
    sdf=cdf.toDF("JURISDICTION_LEVEL","ORDER_ID","TAX_SITUS","IMPOSITION_VALUE","SELLER_ID","IMPOSITION_TYPE","JURISDICTION_ID")
    sdf.show()
    
    return sdf
   

In [63]:
 sdf = prepareExpDFRSimplT2(jsondf,"111",table2)

+----------------+--------------------+--------------------+--------------------+
|customAttributes|               order|           topicName|     order_lineItems|
+----------------+--------------------+--------------------+--------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|
+----------------+--------------------+--------------------+--------------------+

+----------------+--------------------+--------------------+--------------------+-----------------------+
|customAttributes|               order|           topicName|     order_lineItems|order_lineItems_charges|
+----------------+--------------------+--------------------+--------------------+-----------------------+
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|   {[], Price, [], {...|
|          {HTTP}|{[{dest_0, 1000 W...|LVS.UAT01.order_c...|{[], ANY, [{[], P...|   {[], Shipping, []...|
+----------------+--------------------+--------------------

In [65]:
writeToSQLWarehouse(sdf,"111",table2)

KAFKARADIAL.ROM_ORDER_TAX_BREAKUP_STG0
DataFrame[JURISDICTION_LEVEL: string, ORDER_ID: string, TAX_SITUS: string, IMPOSITION_VALUE: string, SELLER_ID: string, IMPOSITION_TYPE: string, JURISDICTION_ID: string]


In [ ]:
def pullCtry(jsonpathval,romlistwithval,colslist,lookupcolumn,lookupkey,tbl):
    
    ############# Pull COUNTRY_OF_ORIGIN ##############
    
    try:
    
      if ( lookupcolumn in colslist ):

          print("Inside*******")
        
          item_id_idx=romlistwithval.index(lookupkey)
          item_id=jsonpathval[item_id_idx]
          print(item_id)
          tableinfo=[tbl,lookupcolumn]
          lookuptablelist.append(tableinfo)
       
    
          ctry=lookupItemID(lookuptablelist, item_id)
          jsonpathval.append(ctry)
          romlistwithval.append(lookupcolumn)
     
    except Exception as e:
          print(e)
        
    
    return jsonpathval, romlistwithval
    
    

In [ ]:
#pullCtry(["100"],["ITEM_ID"],["COUNTRY_OF_ORIGIN","ITEM_ID"],"COUNTRY_OF_ORIGIN","ITEM_ID","KAFKARADIAL.PRODUCT_MASTER")

In [ ]:
####### Writing Steams to DB #######

query1 = df \
    .writeStream \
    .outputMode("append") \
    .option("partition.assignment.strategy", "range")\
    .foreachBatch(popTables)\
    .start()

query1.awaitTermination(10)

In [ ]:
query1.stop()

In [ ]:
query1.status

In [ ]:
elm_up_arr=["1","2","3","4","5"]

In [ ]:
cols_str=','.join(elm_up_arr)

In [ ]:
print(cols_str)